In [4]:
# Import necessary modules
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# from imblearn.over_sampling import SMOTE 
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier, Booster
import pickle

ImportError: cannot import name '_to_object_array' from 'sklearn.utils' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\__init__.py)

In [ ]:
#importing the data into the notebook
df = pd.read_csv('Notebook/Data/diabetes_data.csv') 
df.head()

## Understanding the dataset

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.corr()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
y = df.diabetes
X = df.drop('diabetes', axis=1)
smt = SMOTE()
X,y = smt.fit_resample(X,y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

In [ ]:
# one hot encode outputs



## Creating the model

In [ ]:
model = Pipeline(steps=[
    ('clf',XGBClassifier())
])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
with open("pickle_model.pkl",'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open("pickle_model.pkl", "rb") as file:
    pkl_model = pickle.load(file)
pkl_model.predict(X_test)

In [ ]:
X_test